# Poutyne's tips and tricks

Poutyne also over a variety of tools for fine tuning the information generated during the training, such as a coloring of the training, a progress bar, multi-GPUs, user callbacks interface and a user naming interface for the metrics names. 

## 


#Also, Poutyne use by default a coloring template of the training step when the package `colorama` is installed.
#One could either remove the coloring (`color_log=False`) or set a different coloring template using the fields:
#`text_color`, `ratio_color`, `metric_value_color`, `time_color` and `progress_bar_color`.
#If a field is not specified, the default color will be used.

#Here an example where we set the `text_color` to MAGENTA and the `ratio_color` to BLUE.

#```python
#model.fit(
#    train_x,
#    train_y,
#    validation_data=(valid_x, valid_y),
#    epochs=5,
#    coloring={"text_color": "MAGENTA", "ratio_color":"BLUE"})
#```


In [0]:

#Also, Poutyne use by default a coloring template of the training step when the package `colorama` is installed.
#One could either remove the coloring (`color_log=False`) or set a different coloring template using the fields:
#`text_color`, `ratio_color`, `metric_value_color`, `time_color` and `progress_bar_color`.
#If a field is not specified, the default color will be used.

#Here an example where we set the `text_color` to MAGENTA and the `ratio_color` to BLUE.

#```python
#model.fit(
#    train_x,
#    train_y,
#    validation_data=(valid_x, valid_y),
#    epochs=5,
#    coloring={"text_color": "MAGENTA", "ratio_color":"BLUE"})
#```


# Poutyne Callbacks

One nice feature of Poutyne is [callbacks](https://poutyne.org/callbacks.html). Callbacks allow to do actions during training of the neural network. In the following example, we use 3 callbacks. One that saves the latest weights in a file to be able to continue the optimization at the end of training if more epochs are needed. Another one that saves the best weights according to the performance on the validation dataset. Finally, another one that saves the displayed logs into a TSV file.

In [ ]:
def train_with_callbacks(name, pytorch_network):
    """
    In addition to the the `poutyne_train`, this function saves checkpoints and logs as described above.

    Args:
        name (str): a name used to save logs and checkpoints.
        pytorch_network (torch.nn.Module): The neural network to train.
    """
    print(pytorch_network)

    callbacks = [
        # Save the latest weights to be able to continue the optimization at the end for more epochs.
        ModelCheckpoint(name + '_last_epoch.ckpt', temporary_filename='last_epoch.ckpt.tmp'),

        # Save the weights in a new file when the current model is better than all previous models.
        ModelCheckpoint(name + '_best_epoch_{epoch}.ckpt', monitor='val_acc', mode='max', save_best_only=True, restore_best=True, verbose=True, temporary_filename='best_epoch.ckpt.tmp'),

        # Save the losses and accuracies for each epoch in a TSV.
        CSVLogger(name + '_log.tsv', separator='\t'),
    ]

    train_loader, valid_loader, test_loader = loaders
    optimizer = optim.SGD(pytorch_network.parameters(), lr=learning_rate)
    loss_function = nn.CrossEntropyLoss()
    model = Model(pytorch_network, optimizer, loss_function, batch_metrics=['accuracy'])
    model.to(device)
    model.fit_generator(train_loader, valid_loader, epochs=num_epochs, callbacks=callbacks)
    test_loss, test_acc = model.evaluate_generator(test_loader)
    print('Test:\n\tLoss: {}\n\tAccuracy: {}'.format(test_loss, test_acc))

In [ ]:
fc_net = create_fully_connected_network()
train_with_callbacks('fc', fc_net)

In [ ]:
conv_net = create_convolutional_network()
train_with_callbacks('conv', conv_net)

# Making Your Own Callback

While Poutyne provides a great number of [predefined callbacks](https://poutyne.org/callbacks.html), it is sometimes useful to make your own callback.

In the following example, we want to see the effect of temperature on the optimization of our neural network. To do so, we either increase or decrease the temperature during the optimization. As one can see in the result, temperature either as no effect or has detrimental effect on the performance of the neural network. This is so because the temperature has for effect to artificially changing the learning rates. Since we have found the right learning rate, increasing or decreasing it shows no improvement on the results.

In [ ]:
class CrossEntropyLossWithTemperature(nn.Module):
    """
    This loss module is the cross-entropy loss function
    with temperature. It divides the logits by a temperature
    value before computing the cross-entropy loss.

    Args:
        initial_temperature (float): The initial value of the temperature.
    """
    def __init__(self, initial_temperature):
        super().__init__()
        self.temperature = initial_temperature
        self.celoss = nn.CrossEntropyLoss()

    def forward(self, y_pred, y_true):
        y_pred = y_pred / self.temperature
        return self.celoss(y_pred, y_true)

class TemperatureCallback(Callback):
    """
    This callback multiply the loss temperature with a decay before
    each batch.

    Args:
        celoss_with_temp (CrossEntropyLossWithTemperature): the loss module.
        decay (float): The value of the temperature decay.
    """
    def __init__(self, celoss_with_temp, decay):
        super().__init__()
        self.celoss_with_temp = celoss_with_temp
        self.decay = decay

    def on_train_batch_begin(self, batch, logs):
        self.celoss_with_temp.temperature *= self.decay

def train_with_temperature(pytorch_network, initial_temperature, temperature_decay):
    """
    In addition to the the `poutyne_train`, this function uses a cross-entropy loss
    with temperature and decays the temperature at each batch.

    Args:
        pytorch_network (torch.nn.Module): The neural network to train.
        initial_temperature (float): The initial value of the temperature.
        decay (float): The value of the temperature decay.
    """
    print(pytorch_network)
    train_loader, valid_loader, test_loader = loaders
    optimizer = optim.SGD(pytorch_network.parameters(), lr=learning_rate)

    loss_function = CrossEntropyLossWithTemperature(initial_temperature)
    callbacks = [TemperatureCallback(loss_function, temperature_decay)]
    model = Model(pytorch_network, optimizer, loss_function, batch_metrics=['accuracy'])
    model.to(device)
    model.fit_generator(train_loader, valid_loader, epochs=num_epochs, callbacks=callbacks)
    test_loss, test_acc = model.evaluate_generator(test_loader)
    print('Test:\n\tLoss: {}\n\tAccuracy: {}'.format(test_loss, test_acc))

In [ ]:
conv_net = create_convolutional_network()
# Initial temperature = 0.1
# Final temperature ≈ 0.1 * 1.0008^7500 ≈ 40.25
train_with_temperature(conv_net,
                       initial_temperature=0.1,
                       temperature_decay=1.0008)

In [ ]:
conv_net = create_convolutional_network()
# Initial temperature = 40.25
# Final temperature ≈ 40.25 * 0.9992^7500 ≈ 0.1
train_with_temperature(conv_net,
                       initial_temperature=4.25,
                       temperature_decay=0.9995)